In [1]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
import os
from stable_baselines3 import PPO

from stable_baselines3.common.results_plotter import load_results, ts2xy
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import SubprocVecEnv
import gym

In [2]:
class FrameSkip(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        super(FrameSkip, self).__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = False
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

In [3]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
monitor_dir = r'./monitor_log/'
os.makedirs(monitor_dir,exist_ok=True)
env = Monitor(env,monitor_dir)
env = FrameSkip(env, skip=4)
env = GrayScaleObservation(env,keep_dim=True)

# 使用并行环境

env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')


In [4]:
bast_params={          #最优超参数
    'n_steps': 7168,
    'gamma': 0.8692871366327747,
    'learning_rate': 6.642559213980066e-05,
    'clip_range': 0.31688308594665404,
    'gae_lambda': 0.8710254680014865
}

In [5]:
tensorboard_log = r'./tensorboard_log/'
#model = PPO("CnnPolicy", env, verbose=1,
#            tensorboard_log = tensorboard_log,
#            **bast_params
#            )
model = PPO("CnnPolicy", env, verbose=1, 
            tensorboard_log=tensorboard_log, 
            device='cuda', **bast_params
            )

Using cpu device
Wrapping the env in a VecTransposeImage.


In [6]:
save_model_dir  = r"D:\visual code\RL_Mario1\Final Trainng\best_model\model_420000.zip"

model.set_parameters(save_model_dir)

In [7]:
# 定义回调函数
class SaveOnBestTrainingRewardCallback(BaseCallback):
    def __init__(self, check_freq, log_dir, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            x, y = ts2xy(load_results(self.log_dir), 'timesteps')
            if len(x) > 0:
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}")
                    self.model.save(self.save_path)
        return True



In [8]:
save_model_dir = r"D:\visual code\RL_Mario1\Final Trainng"

callback1 = SaveOnBestTrainingRewardCallback(30000, save_model_dir)#每隔n步保存一次模型
model.learn(total_timesteps=1000000,callback=callback1)#训练总步数
# model.save("mario_model")

Logging to ./tensorboard_log/PPO_30


d:\visual code\.conda\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.75e+03 |
|    ep_rew_mean     | 1.71e+03 |
| time/              |          |
|    fps             | 59       |
|    iterations      | 1        |
|    time_elapsed    | 121      |
|    total_timesteps | 7168     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 3.22e+03  |
|    ep_rew_mean          | 1.96e+03  |
| time/                   |           |
|    fps                  | 15        |
|    iterations           | 2         |
|    time_elapsed         | 913       |
|    total_timesteps      | 14336     |
| train/                  |           |
|    approx_kl            | 2.4105802 |
|    clip_fraction        | 0.482     |
|    clip_range           | 0.317     |
|    entropy_loss         | -0.397    |
|    explained_variance   | 0.59      |
|    learning_rate        | 6.64e-05  |
|    loss           

LoadMonitorResultsError: No monitor files of the form *monitor.csv found in D:\visual code\RL_Mario1\Final Trainng